In [1]:
import os, warnings, sys, yaml
from stream2segment.process.db import get_session
from stream2segment.io.db.models import Station, Segment, concat, Channel
from stream2segment.io.utils import loads_inv
# warnings.simplefilter('always')
_fle = os.path.abspath(os.path.join('.', 'jnconfig.yaml'))
assert os.path.isfile(_fle)
with open(_fle, "r") as _:
    jnconfig = yaml.safe_load(_)

dbpath_old = jnconfig['dbpath_old']
dbpath_new = jnconfig['dbpath_new']

In [2]:
import csv

# read the ids from the file of stations with inventory and data. The query used was:
# sess.query(Station).filter((Station.has_inventory) & (Station.segments.any(Segment.has_data)))
# which translates into:
# SELECT stations.id 
# FROM stations 
# WHERE stations.inventory_xml IS NOT NULL AND length(stations.inventory_xml) > 0 AND (EXISTS (SELECT 1 
# FROM channels, segments 
# WHERE stations.id = channels.station_id AND segments.channel_id = channels.id AND segments.data IS NOT NULL AND length(segments.data) > 0))

sta_ids = []
with open(os.path.join(os.getcwd(), 'stations.ids.with.inventory.and.segments.csv')) as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    sta_ids = [int(_[0]) for _ in csv_reader if _[0] != 'id']

print('Done. %d ids: %s (first 10 ids)' % (len(sta_ids), str(sta_ids[:10])))

def printred(string):
    print("\x1b[31m%s\x1b[0m" % string)

Done. 6641 ids: [3565, 6158, 2026, 4349, 7247, 9545, 6122, 12267, 12244, 12423] (first 10 ids)


In [3]:
from collections import defaultdict

def sensors(inventory):
    for n in inventory:
        for s in n:
            for c in s:
                if c.sensor.type:
                    yield c.sensor.type
                # print(c)
                # print(c.sensor.type)

stas = {}  # a dict of station ids mapped to a set of sensor descriptions

sess = get_session(dbpath_new)
unique_sensor_desc = set()
i = 0
step_i = int(len(sta_ids) / 10)
next_i = step_i
try:
    qry = sess.query(Station).filter(Station.id.in_(sta_ids))
    # print(qry)
    for sta in qry.limit(1000):
        # print(sta.id)
        try:
            inv = sta.inventory()
            for s in sensors(inv):
                if sta.id not in stas:
                    stas[sta.id] = set()
                # print(stas[sta.id])
                stas[sta.id].add(s)
                unique_sensor_desc.add(s)
                # print(stas[sta.id])
                if i > next_i:
                    print("%d done" % int(100 * i / len(sta_ids)))
                    next_i += step_i
                          
        except:
            raise
finally:
    sess.close()

print('Unique sensor descriptions')
for _ in unique_sensor_desc:
    print(_)

print("")
print('Unique sensor descriptions per station id')
for staid, sensors in stas.items():
    for sens in sensors:
        print("%-6d %s" % (staid, str(sens)))

# kinematrics, nanometrics (trillium), guralp, reftec, steckeisen (sts1 sts2) 

/Users/riccardo/work/gfz/projects/sources/python/sod/.env/sod/lib/python3.7/site-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


Unique sensor descriptions
TRILLIUM-Compact broad band triaxial sensor by Na
Microbaro STRECKEISEN 279
EpiSensor accelerometer
MARK PRODUCT L22
02
TRILLIUM-120PA broad band triaxial sensor by Nano
RT
MARK PRODUCT L4C
Noemax seismometer
AGECODAGIS IHR1C
CMG-5
FBA
20
Trillium 120
Speed and direction wind sensor
CMG-SP1 30 s - 80 Hz borehole seismometer
GEOSPACE GS1LF
CMG3-ESP Compact 60 s - 80 Hz seismometer
STS-2 broad band triaxial sensor by Streckeisen
W
EE08 Temperature + Humidity Sensor
BW
LE-3Dlite MkIII, 1 s, 800 V/m/s-Centaur, 40 vpp (g
Temperature sensor + OCA design
CMG5 with high pass filter accelerometer
TRILLIUM COMPACT
TRILLIUM-120QA broad band triaxial sensor by Nano
GURALP CMG40
TRILLIUM-240 broad band triaxial sensor by Nanome
CMG3-ESP broad band triaxial sensor by Guralp
BWR
CMG3-ESPC broad band triaxial sensor by Guralp
KINEMETRICS ESDH
Trillium 40 seismometer
07
CMG-3TB-INCLINO, inclinometric part of the CMG-3T
LE-1DV MkII, 1 s, 400 V/m/s-Centaur, 40 vpp (gain
KINEMET